In [48]:
from heapq import heappop, heappush

In [21]:
'''Grid'''
class Map:

    # 
    def __init__(self):
        '''
        Default constructor
        '''

        self.width = 0
        self.height = 0
        self.cells = []
    

    def SetGridCells(self, width, height, gridCells):
        '''
        Initialization of map by list of cells.
        '''
        self.width = width
        self.height = height
        self.cells = gridCells


    def inBounds(self, i, j):
        '''
        Check if the cell is on a grid.
        '''
        return (0 <= j < self.width) and (0 <= i < self.height)
    

    def Traversable(self, i, j):
        '''
        Check if the cell is not an obstacle.
        '''
        return not self.cells[i][j]


    def GetNeighbors(self, i, j):
        '''
        Get a list of neighbouring cells as (i,j) tuples.
        '''   
        neighbors = [[i, j]]
        delta = [[0, 1], [1, 0], [0, -1], [-1, 0]]

        for d in delta:
            if self.inBounds(i + d[0], j + d[1]) and self.Traversable(i + d[0], j + d[1]):
                neighbors.append((i + d[0], j + d[1]))
        return neighbors

In [53]:
import numpy as np

def read_map_from_movingai_file(path):
    map_file = open(path)
    count = 0
    name = map_file.readline()
    height = int(map_file.readline().split()[1])
    width = int(map_file.readline().split()[1])
    type_ = map_file.readline()
    cells = [[0 for _ in range(width)] for _ in range(height)]
    
    i = 0
    j = 0

    for l in map_file:
        j = 0
        for c in l:
            if c == '.':
                cells[i][j] = 0
            elif c == 'T' or c == '@':
                cells[i][j] = 1
            else:
                continue
            
            j += 1
            
        if j != width:
            raise Exception("Size Error. Map width = ", j, ", but must be", width, "(map line: ", i, ")")
                
        i += 1
        if(i == height):
            break
    
    return (width, height, cells)

def read_tasks_from_movingai_file(path):
    tasks = []
    task_file = open(path)
    for l in task_file:
        new_task = l.split()[4:]
        if len(new_task) != 0:
            tasks.append(list(map(float,new_task)))
    #возвращает числа: координаты начала и конца, длину пути
    return np.array(tasks, int)

In [54]:
w, h, cells = read_map_from_movingai_file('empty-16-16.map')
tasks = read_tasks_from_movingai_file('empty-16-16-even-1.scen')

task_map = Map()
task_map.SetGridCells(w, h, cells)

tasks[:3]

array([[10,  8,  8,  5,  3],
       [ 9, 11,  0,  1, 13],
       [13, 10,  8, 11,  5]])

In [480]:
class HighNode:
    '''
    HighNode class represents a high-level search node

    - vertexCons: vertex constraints of the node
    - edgeCons: edge constraints of the node
    - sol: solution of the node
    - g: cost of the solution of the node 
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 

    '''

    def __init__(self, vertexCons={}, edgeCons={}, sol={}, g=0, h=0, F=None, parent=None, k=0):
        self.vertexCons = vertexCons
        self.edgeCons = edgeCons
        self.sol = sol
        self.g = g
        self.h = h
        self.k = k
        if F is None:
            self.F = g + h
        else:
            self.F = F        
        self.parent = parent
    
    
    def __eq__(self, other):
        return (self.vertexCons == other.vertexCons) and (self.edgeCons == other.edgeCons) and \
               (self.sol == other.sol)
    
    def __lt__(self, other):
        return self.F < other.F or ((self.F == other.F) and (self.h < other.h)) \
        or ((self.F == other.F) and (self.h == other.h))

In [481]:
class LowNode:
    '''
    LowNode class represents a low-level search node

    - i, j: coordinates of corresponding grid element
    - g: g-value of the node 
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 
    '''

    def __init__(self, coord, g=0, h=0, F=None, parent=None):
        self.i = coord[0]
        self.j = coord[1]
        self.g = g
        self.h = h
        if F is None:
            self.F = g + h
        else:
            self.F = F        
        self.parent = parent
    
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j) and (self.g == other.g)
    
    def __lt__(self, other):
        return self.F < other.F or ((self.F == other.F) and (self.h < other.h)) \
        or ((self.F == other.F) and (self.h == other.h))

In [482]:
class MetaAgent:
    '''
    MetaAgent class represents a meta-agent
    
    - agents: set of agents of the meta-agent
    '''
    
    def __init__(self, agents):
        self.agents = agents
        
    def __eq__(self, other):
        return self.agents == other.agents

In [483]:
class MetaConstraint:
    '''
    MetaConstraint class represents a meta-constraint
    
    - agents: subset of agents from meta-agent prohibited to from occupying the vertex v at the timestep t
    - v: vertex = point on the grid
    - t: timestep
    '''
    
    def __init__(self, agents, v, t):
        self.agents = agents
        self.v = v
        self.t = t

In [484]:
class OpenHigh:
    
    def __init__(self):
        self.heap = []
        self.elements = []
    
    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True

    
    def AddNode(self, node : HighNode, *args):
        isIn = False
        
        for nd in self.elements:
            if node.vertexCons == nd.vertexCons and node.edgeCons == nd.edgeCons:
                isIn = True
                if node.g < nd.g:
                    self.elements.remove(nd)
                    self.elements.append(node)
                    heappush(self.heap, node)
                    return
            
        if not isIn:
            self.elements.append(node)
            heappush(self.heap, node)
 

    def GetBestNode(self, CLOSED, *args):        
        best = heappop(self.heap)  
            
        while CLOSED.WasExpanded(best):
            best = heappop(self.heap)
            
        self.elements.remove(best)
        return best

In [485]:
class OpenLow:
    
    def __init__(self):
        self.heap = []
        self.elements = {}
    
    def __iter__(self):
        return iter(self.elements.values())
    
    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True

    def AddNode(self, node : LowNode, *args):
        if self.elements.get((node.i, node.j, node.g)) is None or node < self.elements[(node.i, node.j, node.g)]:
            self.elements[(node.i, node.j, node.g)] = node
            heappush(self.heap, node)
        return

    def GetBestNode(self, CLOSED, *args):
        best = heappop(self.heap)    
        while CLOSED.WasExpanded(best):
            best = heappop(self.heap)
        del self.elements[(best.i, best.j, best.g)]
        return best

In [486]:
class ClosedHigh:
    
    def __init__(self):
        self.elements = []


    def __iter__(self):
        return iter(self.elements)
    

    def __len__(self):
        return len(self.elements)


    def AddNode(self, node : HighNode):
        self.elements.append(node)


    def WasExpanded(self, node : HighNode):
        return node in self.elements

In [487]:
class ClosedLow:
    
    def __init__(self):
        self.elements = {}


    def __iter__(self):
        return iter(self.elements.values())
    

    def __len__(self):
        return len(self.elements)


    def AddNode(self, node : LowNode):
        self.elements[(node.i, node.j, node.g)] = node


    def WasExpanded(self, node : LowNode):
        return not self.elements.get((node.i, node.j, node.g)) is None

In [488]:
def ManhattanDistance(i1, j1, i2, j2):
    return abs(i1 - i2) + abs(j1 - j2)

In [489]:
def MakePath(goal):
    '''
    Creates a path by tracing parent pointers from the goal node to the start node
    It also returns path's length.
    '''

    length = goal.g
    current = goal
    path = []
    while current.parent:
        path.append(current)
        current = current.parent
    path.append(current)
    return path[::-1], length

In [490]:
class AstarTimesteps:
    def __init__(self, gridMap, iStart, jStart, iGoal, jGoal, vertexCons, edgeCons):
        self.vertexCons = vertexCons
        self.edgeCons = edgeCons
        self.CLOSED = ClosedLow()
        self.gridMap = gridMap
        self.iStart = iStart
        self.jStart = jStart
        self.iGoal = iGoal
        self.jGoal = jGoal
        self.OPEN = OpenLow()
        self.path = []
        
    def CheckMove(self, i1, j1, i2, j2, t):
        for obs in self.vertexCons:
            if obs[1] == t + 1 and obs[0] == (i2, j2):
                return False
        
        for obs in self.edgeCons:
            if obs[2] == t + 1 and obs[0] == (i1, j1) and obs[1] == (i2, j2):
                return False
                
        return True
    
    def FindPath(self):
        startNode = LowNode(coord=(self.iStart, self.jStart))
        self.OPEN.AddNode(startNode)
    
        while not self.OPEN.isEmpty():
            s = self.OPEN.GetBestNode(self.CLOSED) 
            self.CLOSED.AddNode(s)       
            if s.i == self.iGoal and s.j == self.jGoal:
                return (True, s, self.OPEN, self.CLOSED)
            for nbr in self.gridMap.GetNeighbors(s.i, s.j):
                if self.CheckMove(s.i, s.j, nbr[0], nbr[1], s.g) and \
                not self.CLOSED.WasExpanded(LowNode(coord=nbr, g=s.g + 1)):
                    nbrNode = LowNode(coord=nbr, g=s.g + 1, h=ManhattanDistance(nbr[0], nbr[1], self.iGoal, self.jGoal), \
                                      parent=s)
                    self.OPEN.AddNode(nbrNode)
        return (False, None, self.OPEN, self.CLOSED)

In [498]:
def CBS(gridMap, Starts, Goals, vertexCons, edgeCons): # vertex/edge- Cons - нужны только при вызове CBS в качестве нижнего уровня MACBS
    root = HighNode()
    OPEN = OpenHigh()
    CLOSED = ClosedHigh()
    agents = list(range(len(Starts)))
    k = 0
    
    for a in agents:
        planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], [], [])
        result = planner.FindPath()
        if result[0]:
            path = MakePath(result[1])
            root.sol[a], _ = path
        else:
            return (False, None, OPEN, CLOSED)
    
    root.g = sum([len(path) for path in root.sol.values()])
    OPEN.AddNode(root)
    k += 1
    
    while not OPEN.isEmpty():
        s = OPEN.GetBestNode(CLOSED)
        CLOSED.AddNode(s)  
        newVertexCons = []
        newEdgeCons = []
        
        for a in agents:
            for b in agents[a + 1 :]:
                for step in range(min(len(s.sol[a]), len(s.sol[b]))):
                    if s.sol[a][step].i == s.sol[b][step].i and s.sol[a][step].j == s.sol[b][step].j:
                        newVertexCons.append((a, b, (s.sol[a][step].i, s.sol[a][step].j), step))
                    if step + 1 < min(len(s.sol[a]), len(s.sol[b])) and \
                       s.sol[a][step].i == s.sol[b][step + 1].i and s.sol[a][step].j == s.sol[b][step + 1].j and \
                       s.sol[a][step + 1].i == s.sol[b][step].i and s.sol[a][step + 1].i == s.sol[b][step].i:
                        newEdgeCons.append((
                            a,
                            b,
                            (s.sol[a][step].i, s.sol[a][step].j),
                            (s.sol[a][step + 1].i, s.sol[a][step + 1].j),
                            step,
                        ))
                        
        if len(newVertexCons) == 0 and len(newEdgeCons) == 0:
            return (True, s, OPEN, CLOSED)
        
        # Сейчас сначала разрешаются вершинные конфликты, потом реберные
        if len(newVertexCons) > 0:
            a, b, (i, j), t = newVertexCons[0]
            
            # Разбиваем CT на ноды A и B, разрешая вершинный конфликт 
            
            tmp = s.vertexCons.copy()
            if a in tmp:
                tmp[a].append(((i, j), t))   
            else:
                tmp[a] = [((i, j), t)]
            
            A = HighNode(
                vertexCons=tmp,
                edgeCons=s.edgeCons,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            ec = []
            if a in A.edgeCons:
                ec = A.edgeCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], A.vertexCons[a], ec)
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                A.sol[a], _ = path
                A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(A):
                    OPEN.AddNode(A)
                k += 1
                
            tmp = s.vertexCons.copy()
            if b in tmp:
                tmp[b].append(((i, j), t))   
            else:
                tmp[b] = [((i, j), t)]
                
            B = HighNode(
                vertexCons=tmp,
                edgeCons=s.edgeCons,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            ec = []
            if b in B.edgeCons:
                ec = B.edgeCons[b]
            
            planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], B.vertexCons[b], ec)
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                B.sol[b], _ = path
                B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(B):
                    OPEN.AddNode(B)
                k += 1
                
        elif len(newEdgeCons) > 0:
            a, b, (i1, j1), (i2, j2), t = newEdgeCons[0]
            
            # Разбиваем CT на ноды A и B, разрешая реберный конфликт 
            
            tmp = s.edgeCons.copy()
            if a in tmp:
                tmp[a].append(((i1, j1), (i2, j2), t))   
            else:
                tmp[a] = [((i1, j1), (i2, j2), t)]
            
            A = HighNode(
                vertexCons=s.vertexCons,
                edgeCons=tmp,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            vc = []
            if a in A.vertexCons:
                vc = A.vertexCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vc, A.edgeCons[a])
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                A.sol[a], _ = path
                A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(A):
                    OPEN.AddNode(A)
                k += 1
                
            tmp = s.edgeCons.copy()
            if b in tmp:
                tmp[b].append(((i1, j1), (i2, j2), t))   
            else:
                tmp[b] = [((i1, j1), (i2, j2), t)]
            
            B = HighNode(
                vertexCons=s.vertexCons,
                edgeCons=tmp,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            vc = []
            if b in B.vertexCons:
                vc = B.vertexCons[b]
            
            planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], vc, B.edgeCons[b])
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                B.sol[b], _ = path
                B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(B):
                    OPEN.AddNode(B)
                k += 1
                
    return (False, None, OPEN, CLOSED)

In [508]:
num = 6

Starts = [(item[0], item[1]) for item in tasks[:num]]
Goals = [(item[2], item[3]) for item in tasks[:num]]

result = CBS(task_map, Starts, Goals, {}, {})

print(result[1].g)
sum([tasks[i][-1] for i in range(num)])

93


52

In [19]:
def MAConflict(node: HighNode, CM, a : MetaAgent, b : MetaAgent):
    vertexFlag = False
    edgeFlag = False
    
    for x in a.agents:
        for y in b.agents:
            for step in range(min(len(s.sol[a][x]), len(s.sol[b][y]))):
                    if s.sol[a][x][step].i == s.sol[b][y][step].i and s.sol[a][x][step].j == s.sol[b][y][step].j:
                        vertexFlag = True
                        CM[x][y] += 1
                        CM[y][x] += 1
                    if s.sol[a][x][step].i == s.sol[b][y][step + 1].i and s.sol[a][x][step].j == s.sol[b][y][step + 1].j \
                       and s.sol[a][x][step + 1].i == s.sol[b][y][step].i and s.sol[a][x][step + 1].i == s.sol[b][y][step].i:
                        edgeFlag = True
                        CM[x][y] += 1
                        CM[y][x] += 1
                        
    return CM, vertexFlag, edgeFlag

In [ ]:
def CheckMerge(CM, Border, a: MetaAgent, b : MetaAgent):
    cnt = 0
    
    for x in a.agents:
        for y in b.agents:
            cnt += CM[x][y]
            
    return cnt > Border

In [ ]:
def Merge(a : MetaAgent, b : MetaAgent, vertexConflicts, edgeConflicts, vertexCons, edgeCons):
    newMetaAgent = MetaAgent(a.agents + b.agents)
    newVertexCons = {}
    newEdgeCons = {}            

In [ ]:
def MACBS(gridMap, Starts, Goals, Border):
    root = HighNode()
    OPEN = OpenHigh()
    CLOSED = ClosedHigh()
    agents = [MetaAgent([i]) for i in range(len(Starts))]
    CM = [[0] * len(agents)] * len(agents) # матрица конфликтов
    
    for a in len(agents):
        planner = AstarTimesteps(gridMap, Starts[a].i, Starts[a].j, Goals[a].i, Goals[a].j, [])
        result = planner.FindPath()
        if result[0]:
            path = MakePath(result[1])
            root.sol[agent[a]] = path
        else:
            return (False, None, OPEN, CLOSED)
    
    OPEN.AddNode(root)

    while not OPEN.isEmpty():
        s = OPEN.GetBestNode(CLOSED)
        CLOSED.AddNode(s)       
        newVertexCons = []
        newEdgeCons = []
        vertexConflicts = []
        edgeConflicts = []
        
        # Обновление CM и проверка наличия конфликтов
        for i, a in enumerate(agents):
            for b in agents[i + 1 :]:
                CM, isVertexConf, isEdgeConf = MAConflict(s, CM, a, b)
                if isVertexConf:
                    vertexConflicts.append((a, b))
                if isEdgeConf:
                    edgeConflicts.append((a, b))
        
        # Слияние
        for i, a in enumerate(agents):
            for b in agents[i + 1 :]:
                if CheckMerge(CM, Border, a, b):
                    Merge(a, b, )                    
                
                        
        if len(newVertexCons) == 0 and len(newEdgeCons) == 0:
            return (True, s, OPEN, CLOSED)